In [ ]:
cd D:\\Saarbrucken\\EDA_Research\\vae-disentanglement\\disentanglement_lib_pl

## Keeping everything else (bs, LR, w_recon, w_sup, prior type etc.) the same we just varied w_kld = 5,10,15,20,25 

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import namedtuple
from matplotlib import cm as mpl_colormaps
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import torch
import torchvision.utils as vutils
import torchvision.transforms.functional as T
from torch.utils.data import DataLoader
from torchvision import transforms

import models
from common.data_loader import DSpritesDataset, ThreeShapesDataset, ContinumDataset
#from common import notebook_utils
from common import utils
#from common.notebook_utils import *
from common import notebook_utils as nbutils
from common.utils import CenteredNorm
from gnncsvae_experiment import GNNCSVAEExperiment
from common.ops import reparametrize
# Change figure aesthetics
%matplotlib inline
sns.set_context('talk', font_scale=1.2, rc={'lines.linewidth': 1.5})
np.set_printoptions(suppress=True,precision=4)

In [ ]:
# these two will be used any time we load a saved checkppoint

PROJECT_ROOT = r"D:\Saarbrucken\EDA_Research\vae-disentanglement"
datapath = os.path.join(PROJECT_ROOT, "datasets")
os.environ['DISENTANGLEMENT_LIB_DATA'] = datapath

ModelParams = namedtuple('ModelParams', ["z_dim", "l_dim", "num_labels" , "in_channels", 
                                        "image_size", "batch_size", "w_recon", "w_kld", "w_sup_reg", "kl_warmup_epochs",
                                         "adjacency_matrix", "loss_terms", "use_loss_weights",
                                        "controlled_capacity_increase", "iterations_c", "max_capacity", "dset_name", 
                                         "num_indept_nodes"
                                        ])

ExpParams = dict(
        in_channels=-1,
        image_size=64,
        LR=1e-4,
        weight_decay=0.0,       
        dataset=None,
        datapath=datapath,
        droplast=True,        
        batch_size=64,
        num_workers=0,
        pin_memory=False,
        seed=123,
        evaluation_metrics=None,
        visdom_on=False,
        save_dir=None,
        max_epochs=1,
        l_zero_reg=False
)

# KLD = 5

In [ ]:
# load the learned model here

algo_name = "GNNBasedConceptStructuredVAE"
checkpoint_path = os.path.join(PROJECT_ROOT, "models", "v8", "version_8", "checkpoints", "pendulum144ep1dim_indept.ckpt")
z_dim = 1
channels = 3
l_dim = 4
num_nodes = 4
num_indept_nodes = 3
dataset = "pendulum"
current_device = torch.device("cuda:0")

# prep params
model_params = ModelParams(
        [z_dim], l_dim, 0, channels, 64, 64, 0.80, 0.15, 1.0, 0,
        os.path.join(PROJECT_ROOT, "adjacency_matrices", f"{dataset}.pkl"),
        ['aux_classification'], False,
        False, 0, 0, "pendulum", num_indept_nodes
)

ExpParams['channels'] = channels

# load model
vae_model_class = getattr(models, algo_name)
vae_model = vae_model_class(model_params)

vae_experiment = GNNCSVAEExperiment.load_from_checkpoint(
            checkpoint_path,
            vae_model=vae_model, 
            params=ExpParams,
            dataset_params=None).to(current_device)

# load data
dataset = get_configured_dataset(dataset)
sample_loader = DataLoader(dataset, batch_size=64, shuffle = False, drop_last=True)

In [ ]:
# backup current direct of logs for this expr
# run to comparable epochs 

# Learned Posterior activations
# Learned Prior activations
# 